### Dependency Setup

In [0]:
!pip install git+https://github.com/openai/whisper.git

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-75a4dp1b
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-75a4dp1b
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 10.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801359 sha256=5f1548f1d0e76a70033452ef0b2daf2b88749911b6c819b9dca50c5a05534741
  Stored in dir

In [0]:
import re
import json
import openai
import pandas as pd
import soundfile as sf
import whisper
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:34<00:00, 90.3MiB/s]


### Getting necessary parameters

In [0]:
with open('config.json', 'r') as json_file:
    config = json.load(json_file)

---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
File <command-4503195805848441>, line 1
----> 1 with open('config.json', 'r') as json_file:
      2     config = json.load(json_file)

File /databricks/python/lib/python3.10/site-packages/IPython/core/interactiveshell.py:284, in _modified_open(file, *args, **kwargs)
    277 if file in {0, 1, 2}:
    278     raise ValueError(
    279         f"IPython won't let you open fd={file} by default "
    280         "as it is likely to crash IPython. If you know what you are doing, "
    281         "you can use builtins' open."
    282     )
--> 284 return io_open(file, *args, **kwargs)

FileNotFoundError: [Errno 2] No such file or directory: 'config.json'

### Helper Functions

In [0]:
def get_transcription(audiofile):
    if audiofile[0:2] in ['na', 'uk']:
        whisper_result = model.transcribe(audiofile, fp16 = False)
        original = whisper_result["text"]
        english = whisper_result["text"]
    else:
        whisper_result = model.transcribe(audiofile, fp16 = False)
        original = whisper_result["text"]
        whisper_result = model.transcribe(audiofile, fp16 = False, task = 'translate')
        english = whisper_result["text"]
    return {'original': original, 'english': english}


def get_audio_duration(audiofile):
    f = sf.SoundFile(audiofile)
    audio_duration = int((f.frames / f.samplerate) // 1)
    return audio_duration


def get_llm_response(transcript):

    conversation = [
        {"role": "system", "content": "You are a market expert who can identify the pros and cons of a product from a text."},
        {"role": "user", "content": f"""Understand the following transcript and extract the following items 
                                        [brand, sub-brand, benefits, problems, product category].
                                        When looking for benefits and problems, consider only those arising out of use of the product.  
                                        Here is the text : {transcript}
                                        Present only the results in a neat JSON format. Always use lists when representing the values.
                                        Bargain detergents are competing products. So ignore the problems and benefits of competing products.
                                        Take a deep breath. Let us solve this rationally.
                                        """}
    ]

    openai.api_key = config['api_key']
    results = openai.Completion.create(
        engine = config['model_name'],
        max_tokens = 200
    )

    response = results['choices'][0]['text'].strip()

    return response


def type_correction(resp):
    for value in ['brand', 'sub_brand', 'product_category', 'benefits', 'problems']:
        x = resp[value]
        if x:
            if type(x) == str:
                resp[value] = [x]
        else:
            resp[value] = ''

    for value in ['benefits', 'problems']:
        resp[f'{value}_count'] = len(resp[value])
        resp[value] = ', '.join(resp[value])

    return resp


def wordcounter(text, word):
    text = re.sub('[^A-Za-z0-9]', ' ', text)
    text = re.sub('\s+', ' ', text)
    words_list = [x.strip() for x in text.split()]
    return words_list.count(word)


def brand_subbrand_correction():
    for brand in corrected_llm_resp['brand']:
        list_of_counts = []
        count = wordcounter(whisper_response['english'], brand)
        list_of_counts.append(count)
        corrected_llm_resp[f'brand_count'] = list_of_counts

    for sub_brand in corrected_llm_resp['sub_brand']:
        list_of_counts = []
        count = wordcounter(whisper_response['english'], sub_brand)
        list_of_counts.append(count)
        corrected_llm_resp[f'sub_brand_count'] = list_of_counts


### Driver

In [0]:
files_list = ['de_001.mp3', 'ph_001.mp3', 'ph_002.mp3', 'ph_003.mp3', 'de_002.mp3', 'de_003.mp3', 'ksa_002.mp3', 'ja_001.mp3', 'ksa_003.mp3', 
              'na_002.mp3', 'uk_001.mp3', 'na_001.mp3', 'uk_002.mp3', 'uk_003.mp3', 'ja_002.mp3', 'uk_004.mp3', 'ja_004.mp3', 'uk_005.mp3', 
              'ja_003.mp3', 'ksa_001.mp3']

all_responses = []

for filename in files_list:
    whisper_response = get_transcription(filename)

    llm_response = get_llm_response(whisper_response['english'])
    corrected_llm_resp = type_correction(llm_response)
    brand_subbrand_correction()

    corrected_llm_resp['original'] = whisper_response['original'].strip()
    corrected_llm_resp['english'] = whisper_response['english'].strip()
    corrected_llm_resp['duration'] = get_audio_duration(filename)
    corrected_llm_resp['filename'] = filename
    all_responses.append(corrected_llm_resp)
    print(filename)
    

de_001.mp3
ph_001.mp3
ph_002.mp3
ph_003.mp3
de_002.mp3
de_003.mp3
ksa_002.mp3
ja_001.mp3
ksa_003.mp3
na_002.mp3
uk_001.mp3
na_001.mp3
uk_002.mp3
uk_003.mp3
ja_002.mp3
uk_004.mp3
ja_004.mp3
uk_005.mp3
ja_003.mp3
ksa_001.mp3


### Export to CSV

In [0]:
df = pd.DataFrame()
for resp in all_responses:
    tdf = pd.DataFrame.from_records(resp, index = [0])
    df = pd.concat([df, tdf], axis = 0, ignore_index = True)
df.to_csv('transcript_extract.csv', index = False)